<a href="https://colab.research.google.com/github/GuangyuanHao/AMGAN/blob/master/assets/code/Lecture2-Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
# simple regression data generating process with linear CEF
def gen_data(n, p, beta):
    X = np.random.normal(0, 1, size=(n, p))
    y = X @ beta + np.random.normal(0, 1, size=(n,))
    return X, y

In [ ]:
n = 100
p = 40
beta = np.zeros(p)
beta[0] = 1 # only the first covariate is relevant
X, y = gen_data(n, p, beta)

In [ ]:
lr = LinearRegression().fit(X, y) # fit OLS
rmse = np.sqrt(np.mean((X @ beta - lr.predict(X))**2)) # calculate in-sample RMSE
rmse

0.5706582967272859

In [ ]:
epsilon = y - lr.predict(X) # calculate residual y - X'beta_hat
# calculate theoretical upper bound (up to constants) on RMSE
np.sqrt(np.mean(epsilon**2)) * np.sqrt(p / n)

0.5090178392415295

In [ ]:
# high-dimensional regime; n=p
n = 100
p = 100
beta = np.zeros(p)
X, y = gen_data(n, p, beta)

In [ ]:
lr = LinearRegression().fit(X, y) # fit OLS
rmse = np.sqrt(np.mean((y - lr.predict(X))**2)) # calculate in-sample RMSE
rmse

1.6924465802485186e-14

We see that in sample RMSE is zero!

In [ ]:
# calculate true approximation error with true CEF
rmse = np.sqrt(np.mean((X @ beta - lr.predict(X))**2))
rmse

0.9212488984468548

We see that approximation error is very large! In sample RMSE was mis-leading

In [ ]:
Xtest, ytest = gen_data(n, p, beta) # generate test data

In [ ]:
rmse = np.sqrt(np.mean((ytest - lr.predict(Xtest))**2)) # calculate RMSE on test data
rmse

14.648422979508537

We see it paints a more accurate picture

In [ ]:
from statsmodels.api import OLS
# let's see the coverage performance of difference adjusted covariance
# estimates when p is comparable to n
n = 100
p = 50
true = .1

cov_hc0 = []
cov_hc1 = []
cov_hc3 = []
for _ in range(100):
    X = np.random.normal(0, 1, size=(n, p))
    y = true * X[:, 0] + np.random.normal(0, 1, size=(n,))
    res = OLS(y, X).fit(cov_type='HC0') # plain heteroskedasticity robust std
    ci = res.conf_int()[0]
    cov_hc0 += [(ci[0] <= true) & (true <= ci[1])]
    res = OLS(y, X).fit(cov_type='HC1') # heteroskedasticity robust std with adjustment n/(n-p)
    ci = res.conf_int()[0]
    cov_hc1 += [(ci[0] <= true) & (true <= ci[1])]
    # more conservative robust std with finite sample adjustment
    # provably is conservative even in high-dimensional regime (i.e p/n = constant)
    # see: Cattaneo, Jansson, Newey, Inference in Linear Regression Models
    # with Many Covariates and Heteroscedasticity
    res = OLS(y, X).fit(cov_type='HC3')
    ci = res.conf_int()[0]
    cov_hc3 += [(ci[0] <= true) & (true <= ci[1])]

In [ ]:
np.mean(cov_hc0)

0.79

In [ ]:
np.mean(cov_hc1)

0.92

In [ ]:
np.mean(cov_hc3)

1.0